### Design Search Autocomplete System
<pre>
Design a search autocomplete system for a search engine. Users may input a sentence (at least one word and end with a special character '#'). For each character they type except '#', you need to return the top 3 historical hot sentences that have prefix the same as the part of sentence already typed. Here are the specific rules:

The hot degree for a sentence is defined as the number of times a user typed the exactly same sentence before.
The returned top 3 hot sentences should be sorted by hot degree (The first is the hottest one). If several sentences have the same degree of hot, you need to use ASCII-code order (smaller one appears first).
If less than 3 hot sentences exist, then just return as many as you can.
When the input is a special character, it means the sentence ends, and in this case, you need to return an empty list.
Your job is to implement the following functions:

The constructor function:

AutocompleteSystem(String[] sentences, int[] times): This is the constructor. The input is historical data. Sentences is a string array consists of previously typed sentences. Times is the corresponding times a sentence has been typed. Your system should record these historical data.

Now, the user wants to input a new sentence. The following function will provide the next character the user types:

List<String> input(char c): The input c is the next character typed by the user. The character will only be lower-case letters ('a' to 'z'), blank space (' ') or a special character ('#'). Also, the previously typed sentence should be recorded in your system. The output will be the top 3 historical hot sentences that have prefix the same as the part of sentence already typed.

 
Example:
Operation: AutocompleteSystem(["i love you", "island","ironman", "i love leetcode"], [5,3,2,2])
The system have already tracked down the following sentences and their corresponding times:
"i love you" : 5 times
"island" : 3 times
"ironman" : 2 times
"i love leetcode" : 2 times
Now, the user begins another search:

Operation: input('i')
Output: ["i love you", "island","i love leetcode"]
Explanation:
There are four sentences that have prefix "i". Among them, "ironman" and "i love leetcode" have same hot degree. Since ' ' has ASCII code 32 and 'r' has ASCII code 114, "i love leetcode" should be in front of "ironman". Also we only need to output top 3 hot sentences, so "ironman" will be ignored.

Operation: input(' ')
Output: ["i love you","i love leetcode"]
Explanation:
There are only two sentences that have prefix "i ".

Operation: input('a')
Output: []
Explanation:
There are no sentences that have prefix "i a".

Operation: input('#')
Output: []
Explanation:
The user finished the input, the sentence "i a" should be saved as a historical sentence in system. And the following input will be counted as a new search.
</pre>

In [4]:
class TrieNode:
    def __init__(self, val = None):
        self.val = val
        self.children = dict()
        self.isSentence = False
        self.freq = 0

In [478]:
class AutocompleteSystem:

    def __init__(self, sentences: List[str], times: List[int]):
        # root of Trie or prefix tree
        self.root = TrieNode()
        
        # current node to indicate 
        # this will be advance as and when user input is received
        self.cur = self.root
        
        # variable to store the sentence user is typing
        self.cur_sen = ""
        
        # build sentence to frequency maps
        # insert all the sentences into Trie
        for idx, sen in enumerate(sentences):
            self.__insert(sen, times[idx])
        
    
    def __insert(self, sen, freq):
        # insert a given sentence into Trie
        cur = self.root
        for char in sen:
            if not char in cur.children:
                cur.children[char] = TrieNode(char)
            
            cur = cur.children[char]

        cur.isSentence = True
        cur.freq += freq
    
    def __getAllSentences(self, c):
        # insert char if not in trie 
        if c not in self.cur.children:
            self.cur.children[c] = TrieNode(c)
        
        # update cur node
        self.cur = self.cur.children[c]
        
        sentences = []
        def helper(node:TrieNode, prefix):
            if node.val:
                prefix += node.val
                
            nonlocal sentences
            if node.isSentence:
                sentences.append((node.freq * -1, prefix))
            
            for child in node.children:
                helper(node.children[child], prefix)
        
        helper(self.cur, "")
        
        # list of frequencies and the full sentences
        sen_freq = [(t[0], self.cur_sen + t[1]) for t in sentences]
        
        # sort the list of tuples based on frequencies and their ascii
        sen_freq.sort(key=lambda x: (x[0], x[1]))
        
        # return just the sentences from the top 3 in the sorted list
        return [t[1] for t in sen_freq[:3]]
        
    def input(self, c: str) -> List[str]:
        if c == "#":
            # set current node flag to indicate it's a sentence
            self.cur.isSentence = True
            self.cur.freq += 1
            
            # reset the current node back to root node
            self.cur = self.root
            
            # reset the current sentece to empty string
            self.cur_sen = ""
            
            return []
        else:
            # get all nodes with the given char 
            sen_list = self.__getAllSentences(c)
            
            # append the current character to the so far seen character
            # this builds the sentence string that user is typing
            self.cur_sen += c
            
            return sen_list

In [483]:
obj = AutocompleteSystem(["i love you","island","iroman","i love leetcode"], [5,3,2,2])  

In [484]:
obj.input("i")

['i love you', 'island', 'i love leetcode']

In [485]:
obj.input(" ")

['i love you', 'i love leetcode']

In [486]:
obj.input("a")

[]

In [487]:
obj.input("#")

[]

In [488]:
obj.input("i")

['i love you', 'island', 'i love leetcode']

In [489]:
obj.input(" ")

['i love you', 'i love leetcode', 'i a']

In [490]:
obj.input("a")

['i a']

In [491]:
obj.input("#")

[]

In [492]:
obj.input("i")

['i love you', 'island', 'i a']